In [ ]:
import os
import logging

import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm



In [ ]:
#Clustering


from __future__ import division
from __future__ import print_function

import itertools

import numpy as np
from sklearn.neighbors import NearestNeighbors


class BaseClustering:

    def __init__(self):
        pass

    def fit_predict(self, X):
        pass

    def score(self, X, y_true):
        y_pred = self.fit_predict(X)
        pairwise_precision = self._calc_pw_precision(y_pred, y_true)
        pairwise_recall = self._calc_pw_recall(y_pred, y_true)
        pairwise_f_measure = 2 * (pairwise_precision * pairwise_recall)\
            / (pairwise_precision + pairwise_recall)
        return pairwise_f_measure, pairwise_precision, pairwise_recall

    @staticmethod
    def _calc_pw_precision(y_pred, y_true):
        unique_clusters = np.unique(y_pred)
        n_pairs = 0
        n_same_class_pairs = 0
        for cluster in unique_clusters:
            sample_indices = np.where(y_pred == cluster)[0]
            combs = np.array(list(itertools.combinations(sample_indices, 2)), dtype=np.int64)
            if not np.any(combs):
                continue
            combs_classes = y_true[combs]
            same_class_pairs = np.where(combs_classes[:, 0] == combs_classes[:, 1])[0]
            n_pairs += len(combs)
            n_same_class_pairs += len(same_class_pairs)
        pw_precision = n_same_class_pairs / n_pairs
        return pw_precision

    @staticmethod
    def _calc_pw_recall(y_pred, y_true):
        unique_classes = np.unique(y_true)
        n_pairs = 0
        n_same_cluster_pairs = 0
        for clss in unique_classes:
            sample_indices = np.where(y_true == clss)[0]
            combs = np.array(list(itertools.combinations(sample_indices, 2)), dtype=np.int64)
            if not np.any(combs):
                continue
            combs_clusters = y_pred[combs]
            same_cluster_pairs = np.where(combs_clusters[:, 0] == combs_clusters[:, 1])[0]
            n_pairs += len(combs)
            n_same_cluster_pairs += len(same_cluster_pairs)
        pw_recall = n_same_cluster_pairs / n_pairs
        return pw_recall


class ROCWClustering(BaseClustering):

    """Approximated rank-order clustering implemented using Chinese Whispers algorithm.

    Using rank-order distances generate a graph, and feed this graph to ChineseWhispers
     algorithm for clustering.
    """

    def __init__(self, k, metric, n_iteration, algorithm):
        super().__init__()
        self.k = k
        self.metric = metric
        self.n_iteration = n_iteration
        self.knn_algorithm = algorithm

    def fit_predict(self, X):
        if len(X) > self.k:
            graph = ROGraph(self.k, self.metric, algorithm=self.knn_algorithm)
        else:
            graph = ROGraph(len(X), self.metric, algorithm=self.knn_algorithm)
        adjacency_mat = graph.generate_graph(X)
        clusterer = ChineseWhispersClustering(self.n_iteration)
        labels = clusterer.fit_predict(adjacency_mat)
        return labels


class ChineseWhispersClustering:

    def __init__(self, n_iteration=5):
        self.n_iteration = n_iteration
        self.adjacency_mat_ = None
        self.labels_ = None

    def fit_predict(self, adjacency_mat):

        """Fits and returns labels for samples"""

        n_nodes = adjacency_mat.shape[0]
        indices = np.arange(n_nodes)
        labels_mat = np.arange(n_nodes)
        for _ in range(self.n_iteration):
            np.random.shuffle(indices)
            for ind in indices:
                weights = adjacency_mat[ind]
                winner_label = self._find_winner_label(weights, labels_mat)
                labels_mat[ind] = winner_label
        self.adjacency_mat_ = adjacency_mat
        self.labels_ = labels_mat
        return labels_mat

    @staticmethod
    def _find_winner_label(node_weights, labels_mat):
        adjacent_nodes_indices = np.where(node_weights > 0)[0]
        adjacent_nodes_labels = labels_mat[adjacent_nodes_indices]
        unique_labels = np.unique(adjacent_nodes_labels)
        label_weights = np.zeros(len(unique_labels))
        for ind, label in enumerate(unique_labels):
            indices = np.where(adjacent_nodes_labels == label)
            weight = np.sum(node_weights[adjacent_nodes_indices[indices]])
            label_weights[ind] = weight
        winner_label = unique_labels[np.argmax(label_weights)]
        return winner_label


class ROGraph:

    def __init__(self, k, metric, algorithm):

        self.k = k
        self.metric = metric
        self.knn_algorithm = algorithm
        self.adjacency_mat_ = None

    @property
    def adjacency_mat(self):
        return self.adjacency_mat_

    def generate_graph(self, X):
        order_lists = self._get_knns(X)
        pw_distances = self._generate_normalized_pw_distances(order_lists)
        adjacency_mat = self._generate_adjacency_mat(pw_distances)
        return adjacency_mat

    def _get_knns(self, X):

        """Generates order lists and absolute distances of k-nearest-neighbors
            for each data point.
        """

        nbrs = NearestNeighbors(n_neighbors=self.k,
                                algorithm=self.knn_algorithm,
                                metric=self.metric).fit(X)
        _, order_lists = nbrs.kneighbors(X)
        return order_lists

    def _generate_normalized_pw_distances(self, order_lists):
        n_samples = len(order_lists)
        combs = itertools.combinations([i for i in range(n_samples)], 2)
        pw_distances = np.zeros((n_samples, n_samples))
        for ind1, ind2 in combs:
            order_list_1, order_list_2 = order_lists[ind1], order_lists[ind2]
            pw_dist = self._calc_pw_dist(ind1, ind2, order_list_1, order_list_2)
            pw_distances[ind1, ind2] = pw_dist
        pw_distances = pw_distances / pw_distances.max()
        pw_distances = pw_distances + pw_distances.T
        return pw_distances

    def _generate_adjacency_mat(self, pw_distances):
        adjacency_mat = self._dist2adjacency(pw_distances)
        self.adjacency_mat_ = adjacency_mat
        return adjacency_mat

    @staticmethod
    def _dist2adjacency(distances):
        mask_mat = np.zeros(distances.shape)
        mask_mat[np.where(distances > 0)] = 1
        adjacency_mat = (1 - distances) * mask_mat
        return adjacency_mat

    def _calc_pw_dist(self, ind_a, ind_b, order_list_a, order_list_b):
        pw_dist = 0.0
        if np.any(np.intersect1d(order_list_a, order_list_b)):
            order_b_in_a, order_a_in_b = self._calc_orders(ind_a, ind_b, order_list_a, order_list_b)
            d_m_ab = self._calc_dm(order_list_a, order_list_b, order_b_in_a)
            d_m_ba = self._calc_dm(order_list_b, order_list_a, order_a_in_b)
            min_of_two = min(order_a_in_b, order_b_in_a)
            pw_dist = (d_m_ab + d_m_ba) / min_of_two
        return pw_dist

    def _calc_orders(self, ind_a, ind_b, order_list_a, order_list_b):
        order_b_in_a = np.where(order_list_a == ind_b)[0]
        if not np.any(order_b_in_a):
            order_b_in_a = self.k
        else:
            order_b_in_a = order_b_in_a[0]
        order_a_in_b = np.where(order_list_b == ind_a)[0]
        if not np.any(order_a_in_b):
            order_a_in_b = self.k
        else:
            order_a_in_b = order_a_in_b[0]
        return order_b_in_a, order_a_in_b

    def _calc_dm(self, order_list_a, order_list_b, order_b_in_a):
        dist = 0
        for i in range(min(self.k, order_b_in_a)):
            sample_index = order_list_a[i]
            if np.any(order_list_b == sample_index):
                dist += 1 / self.k
            else:
                dist += 1
        return dist

In [ ]:
#face utils
# -*- coding: utf-8 -*-
"""Face-related utils needed for clustering faces.
"""


import os
import re
import logging

import tensorflow as tf
import numpy as np
import skimage.io as skio
import skimage.color as skcolor
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import dlib
from collections import OrderedDict
import cv2

import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

class FaceNet:

    """FaceNet class containing the facenet model ready to generate embeddings
    for faces.

    Naming rules:
        self.name : externally given parameters (as argument), and public
         parameters
        self.name_ : internally created, non-public parameters
        self.name() : public methods
        self._name() : internal methods

    Public methods:
        generate_embeddings_from_images: given images, generates 512D embeddings.
        generate_embeddings_from_paths: generates embeddings for images found at
            specified paths.
        plot_2d_embeddings: plots generated embeddings for generated embeddings in 2D space,
            after performing dimension reduction using 'TSNE' method.
        clean: closes the running tensorflow Session.

    Attributes:
        sess_: tensorflow session that contains compiled pretrained facenet model.
    """

    def __init__(self,
                 model_path,
                 batch_size,
                 image_size,
                 do_prewhiten,
                 do_crop):

        """Initialize a FaceNet model.

      
        """

        self.logger_ = self._initialize_logger()
       # self.model_dir_ = os.path.join(os.path.dirname(__file__) + '/models/', model_path)
        self.model_dir_ = '/content/drive/MyDrive/Clustering/models'
        self.logger_.info('Model dir: {}'.format(self.model_dir_))
        self.sess_, self.graph_ = self._load_model()
        self.closed_ = False
        (self.images_placeholder_,
         self.embeddings_tensor_,
         self.phase_train_placeholder_) = self._get_tensors()
        self.batch_size = batch_size
        self.image_size = image_size
        self.do_prewhiten = do_prewhiten
        self.do_crop = do_crop

    @staticmethod
    def _initialize_logger():

        """Initializes a console logger for logging purposes."""

        console_handler = logging.StreamHandler()
        formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
        console_handler.setFormatter(formatter)
        logger = logging.getLogger('FaceNet')
        logger.setLevel(logging.INFO)
        logger.addHandler(console_handler)
        return logger

    def _load_model(self):

        """Loads and compiles pretrained model's graph and associated tensors.

        Returns:
            a tensorflow session that contains the loaded and compiled model.

        Raises:
            ValueError: If pretrained model not found or more than one model exists
                in self.model_dir_.
        """

        try:
            meta_file, ckpt_file = self._get_model_filenames(self.model_dir_)
        except Exception as e:
            self.logger_.error(e)
            raise e

        sess =tf.compat.v1.Session()
        #tf.compat.v1.disable_v2_behavior()

        saver = tf.train.import_meta_graph(os.path.join(self.model_dir_, meta_file), input_map=None)
        saver.restore(sess, os.path.join(self.model_dir_, ckpt_file))
        self.logger_.info("Model loaded. Session is ready.")
        return sess, sess.graph

    def _get_tensors(self):

        """Returns needed tensors: placeholders and embeddings tensor."""

        images_placeholder = self.graph_.get_tensor_by_name("input:0")
        embeddings = self.graph_.get_tensor_by_name("embeddings:0")
        phase_train_placeholder = self.graph_.get_tensor_by_name("phase_train:0")

        return images_placeholder, embeddings, phase_train_placeholder

    @staticmethod
    def _get_model_filenames(model_dir):

        """Searches specified paths for model files.

        Returns:
            Model's meta file and checkpoint file

        Raises:
            ValueError: If model's meta file not found or more than one meta file exists.
        """

        files = os.listdir(model_dir)
        meta_files = [s for s in files if s.endswith('.meta')]
        print(meta_files)
        if len(meta_files) == 0:
            raise ValueError('No meta file found in the model directory (%s)' % model_dir)
        elif len(meta_files) > 1:
            raise ValueError('There should not be more than one meta file in\
             the model directory (%s)'
                             % model_dir)
        meta_file = meta_files[0]
        ckpt = tf.train.get_checkpoint_state(model_dir)
        if ckpt and ckpt.model_checkpoint_path:
            ckpt_file = os.path.basename(ckpt.model_checkpoint_path)
            return meta_file, ckpt_file

        meta_files = [s for s in files if '.ckpt' in s]
        max_step = -1
        for f in files:
            step_str = re.match(r'(^model-[\w\- ]+.ckpt-(\d+))', f)
            if step_str is not None and len(step_str.groups()) >= 2:
                step = int(step_str.groups()[1])
                if step > max_step:
                    max_step = step
                    ckpt_file = step_str.groups()[0]
        return meta_file, ckpt_file

    def generate_embeddings_from_images(self, images):

        """Generate embeddings for given images.

        Args:
            images: array of RGB images.

        Returns:
            array of 512-dimensional embeddings generated for given images.
        """

        preprocessed_images = self._preprocess_images(images)
        emb_array = self._generate_embeddings(preprocessed_images)

        return emb_array

    def generate_embeddings_from_paths(self, image_paths):

        """Generate embeddings for images found in specified paths.

        Args:
            image_paths: list of complete paths to images.

        Returns:
            generated 512-dimensional embeddings for images found on paths.
        """

        embedding_size = self.embeddings_tensor_.get_shape()[1]

        # Run forward pass to calculate embeddings
        self.logger_.info('Calculating embeddings ...')
        nrof_images = len(image_paths)
        nrof_batches_per_epoch = int(np.ceil(1.0 * nrof_images / self.batch_size))
        emb_array = np.zeros((nrof_images, embedding_size))

        for i in range(nrof_batches_per_epoch):
            start_index = i * self.batch_size
            end_index = min((i + 1) * self.batch_size, nrof_images)
            paths_batch = image_paths[start_index: end_index]
            images = self._load_data_from_paths(paths_batch)
            emb_array[start_index: end_index, :] = self._generate_batch_embeddings(images)

        return emb_array

    @staticmethod
    def plot_2d_embeddings(embs, labels):

        """Plots embeddings after dimension reduction.

        For explatory tasks only.

        Args:
            embs: generated embeddings for faces.
            labels: labels for given embeddings.
        """

        transformed = TSNE(n_components=2).fit_transform(embs)
        labels_set = np.unique(labels)

        fig = plt.figure(figsize=(8, 8))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlabel('Transformed dim 1', fontsize=15)
        ax.set_ylabel('Transformed dim 2', fontsize=15)
        ax.set_title('2D TSNE', fontsize=20)
        tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
                     (255, 187, 120), (44, 160, 44), (152, 223, 138),
                     (214, 39, 40), (255, 152, 150), (148, 103, 189),
                     (197, 176, 213), (140, 86, 75), (196, 156, 148),
                     (227, 119, 194), (247, 182, 210), (127, 127, 127),
                     (199, 199, 199), (188, 189, 34), (219, 219, 141),
                     (23, 190, 207), (158, 218, 229)]
        colors = [(i[0] / 255., i[1] / 255., i[2] / 255.)
                  for i in tableau20][: len(labels_set)]

        for label, color in zip(labels_set, colors):
            indices_to_keep = np.where(labels == label)[0]
            samples = transformed[indices_to_keep]
            ax.scatter(samples[:, 0], samples[:, 1], color=color, label=label)

        ax.legend(labels_set)
        ax.grid()
        plt.show()
        return

    def clean(self):

        
        self.sess_.close()
        self.closed_ = True

    def _generate_embeddings(self, images):

        """Generates embeddings for given images.

        Args:
            images: preprocessed images.

        Returns:
            generated embeddings.
        """

        embedding_size = self.embeddings_tensor_.get_shape()[1]

        # Run forward pass to calculate embeddings
        nrof_images = images.shape[0]
        nrof_batches_per_epoch = int(np.ceil(1.0 * nrof_images / self.batch_size))
        emb_array = np.zeros((nrof_images, embedding_size))

        for i in range(nrof_batches_per_epoch):
            start_index = i * self.batch_size
            end_index = min((i + 1) * self.batch_size, nrof_images)
            imgs = images[start_index: end_index]
            emb_array[start_index: end_index, :] = self._generate_batch_embeddings(imgs)

        return emb_array

    def _generate_batch_embeddings(self, batch):

        """One run over a batch, that generates embeddings for given batch.

        Args:
            batch: array of preprocessed images.

        Returns:
            generated embeddings
        """

        feed_dict = {self.images_placeholder_: batch, self.phase_train_placeholder_: False}
        return self.sess_.run(self.embeddings_tensor_, feed_dict=feed_dict)

    def _load_data_from_paths(self, image_paths):

        """Loads images in RGB, and returns preprocessed images.

        Args:
            image_paths: [path1, path2, ...]
            Note: images must be aligned faces.

        Returns:
            preprocessed images.
        """

        nrof_samples = len(image_paths)
        images = np.zeros((nrof_samples, self.image_size, self.image_size, 3))

        for i in range(nrof_samples):
            image_path = os.path.expanduser(image_paths[i])
            img = skio.imread(image_path)
            if not np.any(img):
                print('image not found.')
                continue
            img = self._preprocess_image(image=img)
            images[i, :, :, :] = img
        return images

    def _preprocess_images(self, images):

        """Preprocesses array of images.

        Note: Pass aligned faces of size (182, 182) to this method


         if self.image_size is (160, 160)
         for performance maintaining.

        Args:
            images: array of RGB images.

        Returns:
            preprocessed images: cropped and prewhitened.
        """

        nrof_samples = len(images)
        imgs = np.zeros((nrof_samples, self.image_size, self.image_size, 3))
        for i, img in enumerate(images):
            img = self._preprocess_image(image=img)
            imgs[i, :, :, :] = img
        return imgs

    def _preprocess_image(self, image):

        """Preprocess image for feeding to model.

        Note: Pass aligned face of size (182, 182) to this method if self.image_size is (160, 160)
         for performance maintaining.

        Args:
            image: RGB image to preprocess.

        Returns:
            prewhitened and cropped image.
        """

        if self.do_prewhiten:
            image = self._prewhiten(image)
        if self.do_crop:
            image = self._crop(image, self.image_size)
        return image

    @staticmethod
    def _prewhiten(x):

        """Input image standardization.

        This makes predictions much accurate than using non-whitened images.

        Args:
            x: input image

        Returns:
            whitened image
        """

        # mean = np.mean(x)
        # std = np.std(x)
        # std_adj = np.maximum(std, 1.0 / np.sqrt(x.size))
        # y = np.multiply(np.subtract(x, mean), 1 / std_adj)
        y = (x - 127.5) / 128
        return y

    @staticmethod
    def _crop(image, image_size):

        """Crops given image and returns central (image_size, image_size) region."""

        if image.shape[1] > image_size:
            sz1 = int(image.shape[1] // 2)
            sz2 = int(image_size // 2)
            (h, v) = (0, 0)
            image = image[(sz1 - sz2 + v):(sz1 + sz2 + v),
                          (sz1 - sz2 + h):(sz1 + sz2 + h), :]
        return image


class FaceDetector:

    """A class for detecting faces in images or frames.

    Give image to 'detect_faces' method, and it will return all faces as an array.

    Public methods:
        detect_faces: detects faces in given image.

    Attributes:
        detector_: dlib's HOG-based frontal face detector, if cnn_path is None. else the given model will be used.
    """

    def __init__(self, cnn_path=None):
        if cnn_path is None:
            self.detector_ = dlib.get_frontal_face_detector()
        else:
            self.detector_ = dlib.cnn_face_detection_model_v1(cnn_path)

    def detect_faces(self, image):
        """Detects faces in given image.

        Args:
            image: RGB image

        Returns:
            detected faces: an array of rectangles.
        """

        # resized_image = self._resize_image(image, out_pixels_wide=800)
        # gray = (skcolor.rgb2gray(image) * 255).astype(np.uint8)

        rects = self.detector_(image, 0)
        return rects

    @staticmethod
    def _resize_image(img, out_pixels_wide):
        ratio = out_pixels_wide / img.shape[1]
        dim = (int(out_pixels_wide), int(img.shape[0] * ratio))
        resized = cv2.resize(img, dim, interpolation=cv2.INTER_AREA)
        return resized


class FaceAligner:

    """Aligns faces, use aligned faces for generating embeddings via FaceNet.

    Public methods:
        align: aligns faces found in given image.

    Attributes:
        predictor_: dlib's facial shape predictor (5 landmarks).
        landmarks_idxs_: Positions for output landmarks.
        desired_left_eye: Desired output left eye position, given in a (x, y) tuple.
            Percentages between (0.2, 0.4), With 20% you’ll basically be getting a
            “zoomed in” view of the face, whereas with larger values the face will
            appear more “zoomed out.”
        desired_face_width: Output images width, in pixels.
    """

    def __init__(self,
                 predictor_path,
                 desired_left_eye=(0.35, 0.35),
                 desired_face_width=182):
        """Init a FaceAligner instance.

        Args:
            predictor_path: path to dlib's facial shape predictor (5 landmarks)
            desired_left_eye: Desired output left eye position, given in a (x, y) tuple.
                Percentages between (0.2, 0.4), With 20% you’ll basically be getting a
                “zoomed in” view of the face, whereas with larger values the face will
                appear more “zoomed out.”
            desired_face_width: Output images width, in pixels.
        """

        # predictor_dir = os.path.join(os.path.dirname(__file__) + '/models/shape_predictor/',
        #                              predictor_path)
        self.predictor_ = dlib.shape_predictor(predictor_path)
        FACIAL_LANDMARKS_5_IDXS = OrderedDict([
            ("right_eye", (2, 3)),
            ("left_eye", (0, 1)),
            ("nose", (4,))
        ])
        self.landmarks_idxs_ = FACIAL_LANDMARKS_5_IDXS
        self.desired_left_eye = desired_left_eye
        self.desired_right_eye = (1.0 - self.desired_left_eye[0], self.desired_left_eye[1])
        self.desired_face_width = desired_face_width
        self.desired_face_height = desired_face_width

    def align(self, image, rects):
        """Aligns the faces specified by rects in image.

        Args:
            image: The RGB input image.
            rects: The bounding box rectangles produced by dlib’s HOG face detector.

        Returns:
            a list of aligned faces.
        """

        gray = skcolor.rgb2gray(image).astype(np.uint8)
        outputs = list()
        for rect in rects:
            shape = self.predictor_(gray, rect.rect)
            predicted_coords = self._shape_to_np(shape)

            tform_mat = self._calc_transform_mat(predicted_coords)

            output = self._apply_transformation(image, tform_mat)
            outputs.append(output)

        return outputs

    def _apply_transformation(self, image, M):
        """Applies affine transform to given image using given matrix.

        Args:
            image: RGB image
            M: transformation matrix

        Returns:
            Aligned face
        """

        (w, h) = (self.desired_face_width, self.desired_face_height)
        output = cv2.warpAffine(image, M, (w, h), flags=cv2.INTER_CUBIC)
        return output

    def _calc_transform_mat(self, predicted_coords):
        """Generates transorfmation matrix.

        Args:
            predicted_coords: predicted landmarks' coordinates

        Returns:
            Transorm matrix, should given to warpAffine
        """

        eyes_center, angle, scale = self._calc_mat_params(predicted_coords)
        M = cv2.getRotationMatrix2D(eyes_center, angle, scale)

        tX = self.desired_face_width * 0.5
        tY = self.desired_face_height * self.desired_left_eye[1]
        M[0, 2] += (tX - eyes_center[0])
        M[1, 2] += (tY - eyes_center[1])
        return M

    def _calc_mat_params(self, predicted_coords):
        (left_eye_start, left_eye_end) = self.landmarks_idxs_["left_eye"]
        (right_eye_start, right_eye_end) = self.landmarks_idxs_["right_eye"]

        left_eye_pts = predicted_coords[left_eye_start: left_eye_end + 1]
        right_eye_pts = predicted_coords[right_eye_start: right_eye_end + 1]

        left_eye_center = left_eye_pts.mean(axis=0).astype("int")
        right_eye_center = right_eye_pts.mean(axis=0).astype("int")

        dY = right_eye_center[1] - left_eye_center[1]
        dX = right_eye_center[0] - left_eye_center[0]
        angle = np.degrees(np.arctan2(dY, dX)) - 180

        desired_right_eye_x = 1.0 - self.desired_left_eye[0]

        dist = np.sqrt((dX ** 2) + (dY ** 2))
        desired_dist = (desired_right_eye_x - self.desired_left_eye[0])
        desired_dist *= self.desired_face_width
        scale = desired_dist / dist

        eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2,
                       (left_eye_center[1] + right_eye_center[1]) // 2)
        return eyes_center, angle, scale

    @staticmethod
    def _shape_to_np(shape, dtype="int"):

        # Initialize the list of (x, y)-coordinates
        coords = np.zeros((shape.num_parts, 2), dtype=dtype)

        # loop over all facial landmarks and convert them
        # to a 2-tuple of (x, y)-coordinates
        for i in range(0, shape.num_parts):
            coords[i] = (shape.part(i).x, shape.part(i).y)

        # return the list of (x, y)-coordinates
        return coords

Instructions for updating:
non-resource variables are not supported in the long term


In [ ]:
cd /content/drive/MyDrive/Models

/content/drive/MyDrive/Models


In [ ]:
!wget http://dlib.net/files/mmod_human_face_detector.dat.bz2 http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2 https://drive.google.com/file/d/1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-/view


--2022-08-28 10:18:19--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K  --.-KB/s    in 0.09s   

2022-08-28 10:18:20 (6.99 MB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]

--2022-08-28 10:18:20--  http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
Reusing existing connection to dlib.net:80.
HTTP request sent, awaiting response... 200 OK
Length: 5706710 (5.4M)
Saving to: ‘shape_predictor_5_face_landmarks.dat.bz2’

shape_predictor_5_f 100%[===================>]   5.44M  24.6MB/s    in 0.2s    

2022-08-28 10:18:20 (24.6 MB/s) - ‘shape_predictor_5_face_landmarks.dat.bz2’ saved [5706710/5706710]

--2022-08-28 10:18:20--  https://drive.google.com/file/d/1EXPBSXwTaqrSC0OhUdXN

In [ ]:
#face clustering
# -*- coding: utf-8 -*-
"""Face clustering module.

   This file contains FaceClustering class.
   It uses facenet's pretrained CNN for feature extraction, and uses predefined
    clustering algorithms for performing clustering on images or video files.

   Instantiate an object from this class, tune the model on your own data if
    needed, and enjoy clustered faces.


   You can use the model in context manager mode, but the session will
    terminate at the end of the 'with' block. When using in standard way,
    the background tensorflow session wouldn't terminate in order to avoid
    redundant operations.
"""

import os
import logging

import numpy as np
from sklearn.manifold import TSNE
from sklearn.cluster import DBSCAN
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm



class FaceClustering:

    """Performs clustering on faces, based on embeddings, images or video.


    """

    def __init__(self, config, clustering_alg='dbscan'):

        assert clustering_alg in ('cw', 'dbscan')

        if clustering_alg == 'cw':
            self.clusterer_ = ROCWClustering(k=config.ROCW_K,
                                             metric=config.ROCW_METRIC,
                                             n_iteration=config.ROCW_N_ITERATION,
                                             algorithm=config.ROCW_ALGORITHM)
        else:
            self.clusterer_ = DBSCAN(eps=config.DBSCAN_EPS,
                                     min_samples=config.DBSCAN_MIN_SAMPLES,
                                     metric=config.DBSCAN_METRIC)
        print('clustering algorithm created.')

        self.facenet_model_ = FaceNet(model_path=config.FACENET_MODEL_PATH,
                                      batch_size=config.FACENET_BATCH_SIZE,
                                      image_size=config.FACENET_IMAGE_SIZE,
                                      do_prewhiten=config.FACENET_DO_PREWHITEN,
                                      do_crop=config.FACENET_DO_CROP)
        print('facenet is ready.')

        self.face_detector_ = FaceDetector(cnn_path=config.DLIB_FACE_DETECTOR_PATH)
        print('face detector is ready.')

        self.face_aligner_ = FaceAligner(predictor_path=config.DLIB_FACIAL_SHAPE_PREDICTOR_PATH,
                                         desired_left_eye=config.FACE_ALIGNER_LEFT_EYE,
                                         desired_face_width=config.FACE_ALIGNER_OUTPUT_SIZE)
        print('face aligner is ready.')

        self.valid_images = config.VALID_IMAGES
        self.max_input_height = config.MAX_INPUT_HEIGHT
        self.corr_th_between_frames = config.CORRELATION_THRESHOLD_BETWEEN_FRAMES
        self.vid_sampling_period = config.VIDEO_SAMPLING_PERIOD

    def do_clustering(self, X):

        """Clustering on given samples.

        Args:
            X: an array of arrays, each row is a sample and columns are
             features.

        Returns:
            an array that contains predicted labels for given X.
        """

        labels = self.clusterer_.fit_predict(X)
        return labels

    def do_clustering_on_images(self, images_dir):

        """Clustering on images found on specified folder.

        Finds and aligns faces, saves faces in a new folder called "found_faces" on given path,
         then performs clustering on found faces.

        Args:
            images_dir: folder of images.

        Returns:
            A dictionary that maps each found_face_name to its label.
        """

        faces_dir = os.path.join(images_dir, 'found_faces')
        if not os.path.exists(faces_dir):
            embs, faces = self._detect_align_generate_embs(images_dir)
            if np.any(embs):
                labels = self.clusterer_.fit_predict(embs)
                uniques = np.unique(labels)
                os.mkdir(faces_dir)
                for label in uniques:
                    label_dir = os.path.join(faces_dir, str(label))
                    os.mkdir(label_dir)
                    for ind in np.where(labels == label)[0]:
                        img_path = os.path.join(label_dir, '{}.jpg'.format(ind))
                        self._write_bgr_image(faces[ind], img_path)
                        # skio.imsave(img_path, img_as_ubyte(faces[ind]))
                self.draw_tsne(embs, labels, faces_dir)
                print('results saved on ', faces_dir)
        else:
            print('the found_faces sub-directory already exists at ', images_dir)

    def do_clustering_on_video(self, video_path, num_frames=None):

        """Clustering on given video path.

        Args:
            video_path: complete path to video.
            num_frames: how many frames to read and process from video?

        Returns:
            a dictionary containing cropped faces for each cluster:
                {cluster1: [face1, face2, ...], cluster2: [face1, face2], ...}
        """

        base_dir = os.path.dirname(video_path)
        faces_dir = os.path.join(base_dir, 'found_faces')
        if not os.path.exists(faces_dir):
            embs, faces = self._dag_video(video_path, num_frames)
            if embs:
                labels = self.clusterer_.fit_predict(embs)
                uniques = np.unique(labels)
                os.mkdir(faces_dir)
                for label in uniques:
                    label_dir = os.path.join(faces_dir, str(label))
                    os.mkdir(label_dir)
                    for ind in np.where(labels == label)[0]:
                        img_path = os.path.join(label_dir, '{}.jpg'.format(ind))
                        self._write_bgr_image(faces[ind], img_path)
                        # skio.imsave(img_path, img_as_ubyte(faces[ind]))
                self.draw_tsne(embs, labels, faces_dir)
                print('results saved on ', faces_dir)
        else:
            print('found faces already exists.')

    def show_cluster(self, cluster_dir):

        """Shows all members of given cluster in a single image."""

        face_paths = [os.path.join(cluster_dir, item) for item in os.listdir(cluster_dir) if item.endswith('.jpg')]
        fig = self._plot_faces(face_paths)
        fig.show()

    def _dag_video(self, video_path, num_frames):

        """Detect and align the faces found in given video and generate embeddings for them."""

        vidcap = cv2.VideoCapture(video_path)
        w, h, fps, fc = self._get_vid_specs(vidcap)
        if num_frames is None:
            num_frames = fc

        to_skip = int(self.vid_sampling_period * fps)

        embs = list()
        faces = list()
        last_frame = np.zeros((h, w, 3), dtype=np.int8)
        with tqdm(total=int(num_frames / to_skip)) as pbar:
            for frame_ind in range(num_frames):
                frame = self._read_rgb_frame(vidcap)
                if frame is not None:
                    if frame_ind % to_skip == 0:
                        if np.any(frame):
                            if np.any(last_frame):
                                corr = np.abs(np.corrcoef(last_frame.flatten(), frame.flatten())[0, 1])
                            else:
                                corr = 0
                            if corr < self.corr_th_between_frames:
                                aligned_faces = self._detect_align(frame)
                                if np.any(aligned_faces):
                                    embs.extend(self.facenet_model_.generate_embeddings_from_images(aligned_faces))
                                    faces.extend(aligned_faces)
                                    # for j, face in enumerate(aligned_faces):
                                        # face_path = os.path.join(faces_dir, '{}_{}.jpg'.format(frame_ind, j))
                                        # faces.append(face_path)
                        last_frame = frame.copy()
                        pbar.update(1)
                else:
                    raise Exception('could not read frame {}'.format(frame_ind))

        return embs, faces

    def _detect_align(self, image):

        """Detects faces, aligns and returns them."""

        image_height = image.shape[0]
        # if image_height > self.max_input_height:
        scale_factor = self.max_input_height / image_height
        image = cv2.resize(image, dsize=None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

        rects = self.face_detector_.detect_faces(image)
        if not rects:
            return []
        aligned_faces = self.face_aligner_.align(image, rects)
        return aligned_faces

    def _detect_align_generate_embs(self, images_dir):

        """Detects faces, aligns them and returns the generated embeddings for them."""

        image_paths = [os.path.join(images_dir, img_name)
                       for img_name in os.listdir(images_dir) if img_name.lower().endswith(self.valid_images)]

        if not image_paths:
            print('could not find any image in ', images_dir)
            return None, None

        # face_paths = list()
        embs = list()
        faces = list()
        print('processing ...')
        with tqdm(total=len(image_paths)) as pbar:
            for path in image_paths:
                image = self._read_rgb_image(path)
                aligned_faces = self._detect_align(image)
                if aligned_faces:
                    embs.extend(self.facenet_model_.generate_embeddings_from_images(aligned_faces))
                    faces.extend(aligned_faces)
                    # for j, face in enumerate(aligned_faces):
                    #     face_path = os.path.join(face_folder, '{}_{}.jpg'.format(i, j))
                    #     skio.imsave(face_path, face)
                    #     face_paths.append(face_path)
                pbar.update(1)
        # return face_paths, np.array(embs)
        return embs, faces

    def evaluate(self, X, y_true):

        """Evaluates clustering algorithm on given data.

        Args:
            X: an array of arrays
            y_true: 1D array, true labels for each row of X

        Returns:
            pairwise f-measure
        """

        return self.clusterer_.score(X, y_true)

    def evaluate_on_images(self, images, labels):

        """Evaluates clustering on given images.

        Args:
            images: array of rgb images.
            labels: array of labels for given images.

        Returns:
            pairwise f-measure
        """

        embeddings = list()
        flipped_faces = list()
        for image in images:
            rects = self.face_detector_.detect_faces(image)
            if not rects:
                continue
            aligned_faces = self.face_aligner_.align(image, rects)
            for face in aligned_faces:
                flipped_faces.append(cv2.flip(face, 1))
            embs = self.facenet_model_.generate_embeddings_from_images(aligned_faces)
            embeddings.extend(embs)
        flipped_embs = self.facenet_model_.generate_embeddings_from_images(flipped_faces)
        embeddings.extend(flipped_embs)

        return self.clusterer_.score(embeddings, labels)

    def _plot_faces(self, face_paths):
        dim = int(np.sqrt(len(face_paths))) + 1
        fig, axes = plt.subplots(nrows=dim, ncols=dim, figsize=(18, 18))
        for i, ax in enumerate(axes.flatten()):
            if i >= len(face_paths):
                break
            ax.imshow(self._read_rgb_image(face_paths[i]))
        return fig

    def __enter__(self):

        """Returns model when object enters a "with" block"""

        return self

    def __exit__(self):
        self.facenet_model_.close()

    @staticmethod
    def draw_tsne(embs, labels, face_folder):
        transformed = TSNE(n_components=2).fit_transform(embs)
        labels_set = np.unique(labels)

        fig = plt.figure(figsize=(12, 12))
        ax = fig.add_subplot(1, 1, 1)
        ax.set_xlabel('Transformed dim 1', fontsize=15)
        ax.set_ylabel('Transformed dim 2', fontsize=15)
        ax.set_title('2D TSNE', fontsize=20)
        tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14),
                     (255, 187, 120), (44, 160, 44), (152, 223, 138),
                     (214, 39, 40), (255, 152, 150), (148, 103, 189),
                     (197, 176, 213), (140, 86, 75), (196, 156, 148),
                     (227, 119, 194), (247, 182, 210), (127, 127, 127),
                     (199, 199, 199), (188, 189, 34), (219, 219, 141),
                     (23, 190, 207), (158, 218, 229)]
        colors = [(i[0] / 255., i[1] / 255., i[2] / 255.)
                  for i in tableau20][: len(labels_set)]

        for label, color in zip(labels_set, colors):
            indices_to_keep = np.where(labels == label)[0]
            samples = transformed[indices_to_keep]
            ax.scatter(samples[:, 0], samples[:, 1], color=color, label=label)

        ax.legend(labels_set)
        ax.grid()
        fig.savefig(os.path.join(face_folder, 'tsne.png'))

    @staticmethod
    def _read_rgb_frame(vidcap):
        ret, frame = vidcap.read()
        if ret:
            return cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        else:
            return None

    @staticmethod
    def _get_vid_specs(vidcap):
        width = int(vidcap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vidcap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = vidcap.get(cv2.CAP_PROP_FPS)
        fc = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
        return width, height, fps, fc

    @staticmethod
    def _read_rgb_image(imag_path):
        image = cv2.imread(imag_path)
        return cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    @staticmethod
    def _write_bgr_image(image, path):
        cv2.imwrite(path, cv2.cvtColor(image, cv2.COLOR_RGB2BGR))

In [ ]:
#config
from os.path import normpath, abspath, dirname, realpath, join


class FaceKooConfig:

    ROCW_K = 50
    ROCW_METRIC = 'minkowski'
    ROCW_N_ITERATION = 2
    ROCW_ALGORITHM = 'auto'

    # Higher 'min_samples' or lower 'eps' indicate higher density necessary to form a cluster.
    DBSCAN_EPS = 0.4
    DBSCAN_MIN_SAMPLES = 2
    DBSCAN_METRIC = 'cosine'


    #PROJECT_ROOT = normpath(abspath(dirname(dirname(realpath('')))))
    #PROJECT_ROOT=''
    #MODELS_DIR = join(PROJECT_ROOT, 'models')
    MODELS_DIR='/content/drive/MyDrive/Clustering/models'

    # Download from here: https://drive.google.com/file/d/1EXPBSXwTaqrSC0OhUdXNmKSh9qJUQ55-/view
    FACENET_MODEL_PATH = join(MODELS_DIR, '20180402-114759')
    FACENET_BATCH_SIZE = 32
    FACENET_IMAGE_SIZE = 160
    FACENET_DO_PREWHITEN = True
    FACENET_DO_CROP = True

    # Download dlib's face detector model and unzip it: http://dlib.net/files/mmod_human_face_detector.dat.bz2
    DLIB_FACE_DETECTOR_PATH = join(MODELS_DIR, 'mmod_human_face_detector.dat')

    # Download dlib's facial shape predictor and unzip it:
    # http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2
    DLIB_FACIAL_SHAPE_PREDICTOR_PATH = join(MODELS_DIR, 'shape_predictor_5_face_landmarks.dat')
    FACE_ALIGNER_OUTPUT_SIZE = 182
    FACE_ALIGNER_LEFT_EYE = (0.35, 0.35)

    VALID_IMAGES = ('.jpg', '.png')

    # The face-detector may crash when the input image is so large, when the detector is cnn-based,
    #   i.e. DLIB_FACE_DETECTOR_PATH is not None
    MAX_INPUT_HEIGHT = 1024

    # When processing a video, skip detection on frames that have correlation coefficient of higher than this value vs
    #   the most recent frame
    CORRELATION_THRESHOLD_BETWEEN_FRAMES = 0.985

    # Sample video every () seconds when doing clustering on video
    VIDEO_SAMPLING_PERIOD = 1.0

In [ ]:
!wget http://dlib.net/files/mmod_human_face_detector.dat.bz2

--2022-08-28 10:07:33--  http://dlib.net/files/mmod_human_face_detector.dat.bz2
Resolving dlib.net (dlib.net)... 107.180.26.78
Connecting to dlib.net (dlib.net)|107.180.26.78|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694709 (678K)
Saving to: ‘mmod_human_face_detector.dat.bz2’

mmod_human_face_det 100%[===================>] 678.43K  --.-KB/s    in 0.09s   

2022-08-28 10:07:33 (7.33 MB/s) - ‘mmod_human_face_detector.dat.bz2’ saved [694709/694709]



In [ ]:
import bz2
def unzip_bz2_file(zipped_file_name):
    zipfile = bz2.BZ2File(zipped_file_name)
    data = zipfile.read(size=-1)
    newfilepath = zipped_file_name[:-4] #discard .bz2 extension
    open(newfilepath, 'wb').write(data)

In [ ]:
!bunzip2 /content/drive/MyDrive/Models/shape_predictor_5_face_landmarks.dat.bz2


In [ ]:
import cv2
import dlib
import skimage.transform as sktrans

#from src import FaceKooConfig, FaceClusering
config = FaceKooConfig()
config.MAX_INPUT_HEIGHT = 800
facekoo = FaceClustering(config)



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
images_dir = '/content/drive/MyDrive/Models/Images'
facekoo.do_clustering_on_images(images_dir)

In [ ]:
!wget https://github.com/eonr/ShowSegmentation/tree/master/W1%262-Keyshots/SAA_clip_key_frames

--2022-08-28 10:56:50--  https://github.com/eonr/ShowSegmentation/tree/master/W1%262-Keyshots/SAA_clip_key_frames
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘SAA_clip_key_frames’

SAA_clip_key_frames     [ <=>                ] 224.39K  --.-KB/s    in 0.05s   

2022-08-28 10:56:50 (4.05 MB/s) - ‘SAA_clip_key_frames’ saved [229776]



In [ ]:
!unzip /content/drive/MyDrive/Models/20180402-114759.zip

Archive:  /content/drive/MyDrive/Models/20180402-114759.zip
   creating: 20180402-114759/
  inflating: 20180402-114759/model-20180402-114759.meta  
  inflating: 20180402-114759/20180402-114759.pb  
  inflating: 20180402-114759/model-20180402-114759.ckpt-275.data-00000-of-00001  
  inflating: 20180402-114759/model-20180402-114759.ckpt-275.index  


In [ ]:
!unzip /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2

Archive:  /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2 or
        /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2.zip, and cannot find /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2.ZIP, period.


In [ ]:
!tar -xf /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2 

tar: This does not look like a tar archive
tar: Skipping to next header
tar: Exiting with failure status due to previous errors


In [ ]:
!bzip2 -d /content/drive/MyDrive/Models/mmod_human_face_detector.dat.bz2 